Fuente: https://www.iteramos.com/pregunta/80313/como-puedo-acceder-leer-escribir-a-las-hojas-de-calculo-de-google-sheets-con-python

In [ ]:
#   Instalación paquete pygsheets, última versión
pip install https://github.com/nithinmurali/pygsheets/archive/master.zip

In [ ]:
import pandas as pd
import pygsheets
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials


SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = ''
SHEET_ID = ''

# For the full list of dimensions & metrics, check https://developers.google.com/analytics/devguides/reporting/core/dimsmets
DIMENSIONS = ['ga:source','ga:medium']
METRICS = ['ga:users','ga:sessions']

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics


def get_report(analytics):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression':i} for i in METRICS],
          'dimensions': [{'name':j} for j in DIMENSIONS]
        }]
      }
  ).execute()


def convert_to_dataframe(response):
    
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = [i.get('name',{}) for i in columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])]
    finalRows = []
    

    for row in report.get('data', {}).get('rows', []):
      dimensions = row.get('dimensions', [])
      metrics = row.get('metrics', [])[0].get('values', {})
      rowObject = {}

      for header, dimension in zip(dimensionHeaders, dimensions):
        rowObject[header] = dimension
        
        
      for metricHeader, metric in zip(metricHeaders, metrics):
        rowObject[metricHeader] = metric

      finalRows.append(rowObject)
      
      
  dataFrameFormat = pd.DataFrame(finalRows)    
  return dataFrameFormat      
        
 
def export_to_sheets(df):
    gc = pygsheets.authorize(service_file='client_secrets.json')
    sht = gc.open_by_key(SHEET_ID)
    wks = sht.worksheet_by_title('Sheet1')
    wks.set_dataframe(df,'A1')


def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  df = convert_to_dataframe(response)   #df = pandas dataframe
  export_to_sheets(df)                  #outputs to spreadsheet. comment to skip
  print(df)                             

if __name__ == '__main__':
  main()

  Parece que dando permisos a la API de Google Drive, he solucionado el error 403 que no me permitía el acceso.

In [11]:
import pygsheets

client = pygsheets.authorize()

# Open the spreadsheet and the first sheet.
sh = client.open('Activos')
wks = sh.sheet1

# Update a single cell.
wks.update_value('A1', "Numbers on Stuff")

lista = [5,3,8]
# Update the worksheet with the numpy array values. Beginning at cell 'A2'.
#wks.update_values('A2', lista)

# Share the sheet with your friend. (read access only)
sh.share('ralleuc@gmail.com')
# sharing with write access
sh.share('ralleuc@gmail.com', role='writer')